In [2]:
from ortools.linear_solver import pywraplp
import random

In [3]:
schedule = pywraplp.Solver.CreateSolver('SCIP')
P,D,T,S = 3,5,8,5

dic = {}

for p in range(P):
    dic[p] = {}
    for d in range(D):
        dic[p][d] = {}
        for t in range(T):
            dic[p][d][t] = {}
            for s in range(S):
                dic[p][d][t][s] = schedule.BoolVar('x[%i][%i][%i][%i]' % (p,d,t,s))
                
def dic(p,d,t,s):
    return dic[p][d][t][s]

In [4]:
colab = {}
Availability = 75

L = []
H = []

Choice = [True] * Availability + [False] * (100-Availability)

C = 15